In [1]:
import pandas as pd
import numpy as np
import os
import re
import string
import gc
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.tokenize import word_tokenize, sent_tokenize
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import xgboost as xgb

In [22]:
train=pd.read_csv('F:/hackthons/Hacker earth/Effectiveness of STD drug/train.csv')
test=pd.read_csv('F:/hackthons/Hacker earth/Effectiveness of STD drug/test.csv')

In [3]:
train.head(3)

,patient_id,name_of_drug,use_case_for_drug,review_by_patient,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,base_score
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9,20-May-12,27,8.022969
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8,27-Apr-10,192,7.858458
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5,14-Dec-09,17,6.341969


Adding Sentiment Intensity to Review

In [23]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
train = train.join(train['review_by_patient'].apply(lambda x: analyzer.polarity_scores(x)).apply(pd.Series))
test = test.join(test['review_by_patient'].apply(lambda x: analyzer.polarity_scores(x)).apply(pd.Series))

In [24]:
def clean_text(text):
    text=text.lower() 
    text=re.sub(r"\d+","",text) 
    text=text.translate(str.maketrans("","",string.punctuation))  
    text=re.sub('\s+',' ',text)  
    text=[token for token in text.split() if len(token)>2]   
    text=' '.join(text)
    shortword=re.compile(r'\W*\b\w{10,}\b')   
    text=shortword.sub('',text)
    stop_words=set(stopwords.words('english')) 
    tokens=word_tokenize(text)
    ps=PorterStemmer()
    
    text=''.join(text)
    return text

In [25]:
train['Cleaned_Text'] = train['review_by_patient'].apply(clean_text)
test['Cleaned_Text'] = test['review_by_patient'].apply(clean_text)

In [27]:
from nltk.stem import PorterStemmer
ps=PorterStemmer()

In [59]:
stopword = set(stopwords.words('english'))
def convert(words) :
  
    current_words = []
    stop_words=set(stopwords.words('english')) 
    for i in words :
        if i not in stopword :
            
            updated_word = ps.stem(i)
            current_words.append(updated_word.lower())
    return current_words

In [33]:
new = train['drug_approved_by_UIC'].str.split("-", n = 2, expand = True) 
train['Day'] = new[0]
train['Month'] = new[1]
train['Year'] = new[2]
del new

In [34]:
new = test['drug_approved_by_UIC'].str.split("-", n = 2, expand = True) 
test['Day'] = new[0]
test['Month'] = new[1]
test['Year'] = new[2]
del new

In [35]:
train['Drug'] = train['name_of_drug']
train['Drug'] = train['Drug'].str.split('/').str[0]

In [36]:
test['Drug'] = test['name_of_drug']
test['Drug'] = test['Drug'].str.split('/').str[0]

In [37]:
train.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug'],
      dtype='object')

In [38]:
test.columns

Index(['patient_id', 'name_of_drug', 'review_by_patient',
       'drug_approved_by_UIC', 'number_of_times_prescribed',
       'use_case_for_drug', 'effectiveness_rating', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug'],
      dtype='object')

In [39]:
Train_Drugs_Eff_Rating = pd.DataFrame()
Train_Drugs_Eff_Rating['Drug'] = train['Drug']
Train_Drugs_Eff_Rating['Year'] = train['Year']
Train_Drugs_Eff_Rating['Avg_Rating'] = train['effectiveness_rating']

In [40]:
Test_Drugs_Eff_Rating = pd.DataFrame()
Test_Drugs_Eff_Rating['Drug'] = test['Drug']
Test_Drugs_Eff_Rating['Year'] = test['Year']
Test_Drugs_Eff_Rating['Avg_Rating'] = test['effectiveness_rating']

In [41]:
Train_Drugs_Eff_Rating = Train_Drugs_Eff_Rating.groupby(['Drug', 'Year'])['Avg_Rating'].mean().round(2)
Train_Drugs_Eff_Rating = Train_Drugs_Eff_Rating.reset_index()

In [42]:
Test_Drugs_Eff_Rating = Test_Drugs_Eff_Rating.groupby(['Drug', 'Year'])['Avg_Rating'].mean().round(2)
Test_Drugs_Eff_Rating = Test_Drugs_Eff_Rating.reset_index()

In [43]:
train = pd.merge(train, Train_Drugs_Eff_Rating, on = (['Drug', 'Year']), how = 'left')

In [44]:
test = pd.merge(test, Test_Drugs_Eff_Rating, on = (['Drug', 'Year']), how = 'left')

In [45]:
Test_Patient_Id = test['patient_id']

In [46]:
train.columns

Index(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
       'effectiveness_rating', 'drug_approved_by_UIC',
       'number_of_times_prescribed', 'base_score', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug',
       'Avg_Rating'],
      dtype='object')

In [47]:
test.columns

Index(['patient_id', 'name_of_drug', 'review_by_patient',
       'drug_approved_by_UIC', 'number_of_times_prescribed',
       'use_case_for_drug', 'effectiveness_rating', 'neg', 'neu', 'pos',
       'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year', 'Drug',
       'Avg_Rating'],
      dtype='object')

In [48]:
train.drop(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
            'drug_approved_by_UIC', 'Drug'], axis =1, inplace = True)
test.drop(['patient_id', 'name_of_drug', 'use_case_for_drug', 'review_by_patient',
           'drug_approved_by_UIC', 'Drug'], axis =1, inplace = True)

In [49]:
train.columns

Index(['effectiveness_rating', 'number_of_times_prescribed', 'base_score',
       'neg', 'neu', 'pos', 'compound', 'Cleaned_Text', 'lemma', 'Day',
       'Month', 'Year', 'Avg_Rating'],
      dtype='object')

In [50]:
test.columns

Index(['number_of_times_prescribed', 'effectiveness_rating', 'neg', 'neu',
       'pos', 'compound', 'Cleaned_Text', 'lemma', 'Day', 'Month', 'Year',
       'Avg_Rating'],
      dtype='object')

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer(analyzer='word', stop_words='english', token_pattern='(?u)\\b\\w\\w+\\b', 
                             ngram_range=(1,2), max_df=0.9, min_df=0.01, max_features=None, 
                             binary=False, norm='l2', use_idf=1, smooth_idf=1, sublinear_tf=False)

In [29]:
train_tfidf = tfidf_vect.transform(train['Cleaned_Text'])
test_tfidf = tfidf_vect.transform(test['Cleaned_Text'])

In [30]:
train_tfidf.shape, train.shape

((32165, 549), (32165, 12))

In [31]:
train_df = pd.DataFrame(train_tfidf.toarray(),columns=tfidf_vect.get_feature_names()) 

In [32]:
test_df = pd.DataFrame(test_tfidf.toarray(),columns=tfidf_vect.get_feature_names())

In [33]:
target = train['base_score']

In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt
#rmse = sqrt(mean_squared_error(y_actual, y_predicted))
def rmsle(y_true, y_pred):
    return 100*max(0,1-sqrt(mean_squared_error(y_true, y_pred)))

In [35]:
train_df['effectiveness_rating'] = train['effectiveness_rating']
train_df['number_of_times_prescribed'] = train['number_of_times_prescribed']
train_df['Pos_Score'] = train['pos']
train_df['Neg_Score'] = train['neg']
train_df['Neu_Score'] = train['neu']
train_df['compound'] = train['compound']
train_df['Avg_Rating'] = train['Avg_Rating']

In [36]:
test_df['effectiveness_rating'] = test['effectiveness_rating']
test_df['number_of_times_prescribed'] = test['number_of_times_prescribed']
test_df['Pos_Score'] = test['pos']
test_df['Neg_Score'] = test['neg']
test_df['Neu_Score'] = test['neu']
test_df['compound'] = test['compound']
test_df['Avg_Rating'] = test['Avg_Rating']

In [37]:
train_df.shape, test_df.shape

((32165, 556), (10760, 556))

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(train_df, target, test_size=0.20, random_state=1)

In [40]:
X_train.shape, X_cv.shape

((25732, 556), (6433, 556))

In [145]:
import xgboost
xgb = xgboost.XGBRegressor(objective ='reg:linear', n_estimators=1800, learning_rate=0.001, gamma=0, subsample=0.9,
                           colsample_bytree=0.9, max_depth=22)
xgb.fit(X_train,y_train)

C:\Users\hp\Anaconda3\3.7\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[00:27:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.9, gamma=1,
             importance_type='gain', learning_rate=0.001, max_delta_step=0,
             max_depth=22, min_child_weight=1, missing=None, n_estimators=1800,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=1)

In [146]:
xgb_y_tpred = xgb.predict(X_train)
xgb_y_pred = xgb.predict(X_cv)

In [147]:
print('Train rmse of prediction is:', rmsle(y_train, xgb_y_tpred))
print('Test rmse of prediction is:', rmsle(y_cv, xgb_y_pred))

Train rmse of prediction is: 0
Test rmse of prediction is: 0


In [148]:
Predictions_XGB = xgb.predict(test_df)

In [58]:
Submission = pd.DataFrame()
Submission['patient_id'] = Test_Patient_Id
Submission['base_score'] = Predictions

In [59]:
Submission.to_csv('STD_Preds.csv', index=False)

In [102]:
Submission.shape, test_df.shape

((10760, 2), (10760, 603))